In [1]:
from b_get_data import *
from c_model_related import CustomStandardScaler as Cscl
from c_model_related import Trainer as tr
from c_model_related import metrics_create_df
import joblib

In [2]:
gen_features_drop = [
    'Gen_RPM_Max', 'Gen_RPM_Min', 'Gen_Phase1_Temp_Avg', 'Gen_Phase3_Temp_Avg',
    'Amb_WindSpeed_Est_Avg', 'Grd_RtrInvPhase1_Temp_Avg',
    'Grd_RtrInvPhase3_Temp_Avg', 'Rtr_RPM_Max', 'Rtr_RPM_Min',
    'Blds_PitchAngle_Max', 'Blds_PitchAngle_Min',
    'Prod_LatestAvg_ReactPwrGen1', 'Cont_Hub_Temp_Avg', 'Spin_Temp_Avg',
    'Rtr_RPM_Std', 'Rtr_RPM_Avg', 'Cont_VCP_Temp_Avg'
]

gen_bear_features_drop = [
    'Gen_RPM_Max', 'Gen_RPM_Min', 'Gen_Phase1_Temp_Avg', 'Gen_Phase3_Temp_Avg',
    'Amb_WindSpeed_Est_Avg', 'Grd_RtrInvPhase1_Temp_Avg',
    'Grd_RtrInvPhase3_Temp_Avg', 'Rtr_RPM_Max', 'Rtr_RPM_Min',
    'Blds_PitchAngle_Max', 'Blds_PitchAngle_Min',
    'Prod_LatestAvg_ReactPwrGen1', 'Cont_Hub_Temp_Avg', 'Spin_Temp_Avg',
    'Rtr_RPM_Std', 'Rtr_RPM_Avg', 'Cont_VCP_Temp_Avg'
]

hyd_features_drop = [
    'Rtr_RPM_Max', 'Rtr_RPM_Min', 'Blds_PitchAngle_Max', 'Blds_PitchAngle_Min',
    'Blds_PitchAngle_Max', 'Grd_RtrInvPhase3_Temp_Avg', 'Grd_Busbar_Temp_Avg',
    'Amb_WindSpeed_Est_Avg', 'Spin_Temp_Avg', 'Cont_Hub_Temp_Avg',
    'Grd_RtrInvPhase1_Temp_Avg', 'Cont_VCP_Temp_Avg'
]

gearbox_features_drop = [
    'Rtr_RPM_Max', 'Rtr_RPM_Min', 'Grd_RtrInvPhase1_Temp_Avg',
    'Grd_RtrInvPhase3_Temp_Avg', 'Blds_PitchAngle_Min', 'Blds_PitchAngle_Max',
    'Cont_VCP_Temp_Avg', 'Grd_Busbar_Temp_Avg', 'Amb_WindSpeed_Est_Avg',
    'Spin_Temp_Avg'
]

transf_features_drop = [
    'HVTrafo_Phase1_Temp_Avg', 'HVTrafo_Phase3_Temp_Avg', 'Rtr_RPM_Max',
    'Rtr_RPM_Min', 'Grd_RtrInvPhase1_Temp_Avg', 'Grd_RtrInvPhase3_Temp_Avg',
    'Blds_PitchAngle_Min', 'Blds_PitchAngle_Max', 'Amb_WindSpeed_Est_Avg',
    'Spin_Temp_Avg', 'Cont_VCP_Temp_Avg'
]


feat_drop_list = [
    gen_features_drop, hyd_features_drop, gen_bear_features_drop,
    transf_features_drop, gearbox_features_drop
]

In [3]:
# Obter o caminho dos ficheiros.
root_dir = os.path.abspath('..')
csv_path = os.path.join(root_dir, 'rawdata')
# Importar o dataset de failures
failures_path = os.path.join(csv_path, 'wind-farm-1-failures-training.csv')
failures_df = get_data(failures_path)
# Importar o dataset de signals
signals_path = os.path.join(csv_path, 'wind-farm-1-signals-training.csv')
signals_df = get_data(signals_path)
# Cortar colunas que não têm valores
cols_to_drop = ['Prod_LatestAvg_ActPwrGen2', 'Prod_LatestAvg_ReactPwrGen2']
signals_df = signals_df.drop(columns=cols_to_drop)
signals_df

,Turbine_ID,Timestamp,Gen_RPM_Max,Gen_RPM_Min,Gen_RPM_Avg,Gen_RPM_Std,Gen_Bear_Temp_Avg,Gen_Phase1_Temp_Avg,Gen_Phase2_Temp_Avg,Gen_Phase3_Temp_Avg,...,Grd_Prod_PsbleInd_Avg,Grd_Prod_PsbleInd_Max,Grd_Prod_PsbleInd_Min,Grd_Prod_PsbleInd_Std,Grd_Prod_PsbleCap_Avg,Grd_Prod_PsbleCap_Max,Grd_Prod_PsbleCap_Min,Grd_Prod_PsbleCap_Std,Gen_Bear2_Temp_Avg,Nac_Direction_Avg
0,T11,2016-01-01 00:00:00,1339.4,1233.3,1270.9,18.8,42.0,59,61,60,...,-973.4,-752.6,-1000.0,56.5,973.4,1000.0,752.6,56.5,38,206.9
1,T06,2016-01-01 00:00:00,1270.0,1232.8,1248.5,6.8,42.0,51,52,53,...,-307.1,0.0,-880.6,180.0,307.1,880.6,0.0,180.0,35,204.6
2,T01,2016-01-01 00:00:00,1277.4,1226.1,1249.0,9.0,41.0,58,59,58,...,-144.4,0.0,-584.5,157.1,144.4,584.5,0.0,157.1,37,218.5
3,T09,2016-01-01 00:00:00,1376.7,1234.3,1272.0,24.7,40.0,58,57,58,...,-861.6,-224.5,-1000.0,185.5,861.6,1000.0,224.5,185.5,33,214.0
4,T07,2016-01-01 00:00:00,1317.5,1229.5,1254.9,13.8,41.0,62,62,61,...,-536.5,0.0,-1000.0,338.9,536.5,1000.0,0.0,338.9,39,197.3
5,T09,2016-01-01 00:10:00,1485.0,1239.7,1321.4,50.4,40.0,58,57,58,...,-871.4,-293.3,-1000.0,227.4,871.4,1000.0,293.3,227.4,33,214.0
6,T07,2016-01-01 00:10:00,1271.1,1233.4,1251.9,7.1,42.0,62,62,62,...,-469.6,0.0,-763.5,180.8,469.6,763.5,0.0,180.8,39,203.9
7,T06,2016-01-01 00:10:00,1429.7,203.9,1154.1,271.7,42.0,51,52,52,...,-70.5,0.0,-1000.0,154.1,70.5,1000.0,0.0,154.1,35,210.5
8,T01,2016-01-01 00:10:00,1268.3,93.3,999.7,435.9,41.0,58,59,59,...,-79.6,0.0,-501.0,123.7,79.6,501.0,0.0,123.7,37,218.5
9,T11,2016-01-01 00:10:00,1400.5,1239.8,1303.6,40.2,43.0,59,61,60,...,-999.3,-931.3,-1000.0,5.8,999.3,1000.0,931.3,5.8,38,206.9


In [4]:
# print('002 - Criar o dicionário com os Dataframes originais')
df_dict = {'failures_df':failures_df, 'signals_df':signals_df}

In [5]:
df_dict.keys()

dict_keys(['failures_df', 'signals_df'])

In [6]:
df_generator, df_gen_bear, df_transformer, df_hydraulic, df_gearbox = component_df_creation(
    signals_df)

In [7]:
comp_df_dict = {
    'df_generator': df_generator,
    'df_hydraulic': df_hydraulic,
    'df_gen_bear': df_gen_bear,
    'df_transformer': df_transformer,
    'df_gearbox': df_gearbox
}

In [8]:
for key in comp_df_dict:
    print(comp_df_dict[key].shape)

(434145, 45)
(434145, 34)
(434145, 45)
(434145, 36)
(434145, 33)


In [9]:
comp_prep_df_dict = comp_df_dict.copy()

In [10]:
component_list = [
    'GENERATOR', 'HYDRAULIC_GROUP', 'GENERATOR_BEARING', 'TRANSFORMER',
    'GEARBOX'
]
for i, key in enumerate(comp_prep_df_dict):
    comp_prep_df_dict[key] = sig_fail_merge_dfs(sig_df=comp_prep_df_dict[key],
                                                fail_df=failures_df,
                                                component=component_list[i])

In [11]:
for key in comp_prep_df_dict:
    print(comp_prep_df_dict[key].shape)

(434147, 46)
(434145, 35)
(434145, 46)
(434145, 37)
(434145, 34)


In [12]:
turbine_list = ['T11', 'T06', 'T01', 'T09', 'T07']
for i, key in enumerate(comp_prep_df_dict):
    comp_prep_df_dict[key] = fill_na_by_turbine(comp_prep_df_dict[key],
                                                turbine_list)

C:\Users\ricar\code\RicardoJPedro82\wmillfailprev\wmillfailprev\b_get_data.py:110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  index['date'] = index['Timestamp']
C:\Users\ricar\code\RicardoJPedro82\wmillfailprev\wmillfailprev\b_get_data.py:128: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  df_ = pd.concat([df_, df_final])
C:\Users\ricar\code\RicardoJPedro82\wmillfailprev\wmillfailprev\b_get_data.py:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


In [13]:
for key in comp_prep_df_dict:
    print(comp_prep_df_dict[key].shape)

(434147, 47)
(434145, 36)
(434145, 47)
(434145, 38)
(434145, 35)


In [14]:
for key in comp_prep_df_dict:
    print(comp_prep_df_dict[key].isna().sum().value_counts(), key)

0    47
dtype: int64 df_generator
0    36
dtype: int64 df_hydraulic
0    47
dtype: int64 df_gen_bear
0    38
dtype: int64 df_transformer
0    35
dtype: int64 df_gearbox


In [15]:
for key in comp_prep_df_dict:
    comp_prep_df_dict[key] = aplic_var_target(comp_prep_df_dict[key], 60)

In [16]:
for key in comp_prep_df_dict:
    print(comp_prep_df_dict[key].shape, key)

(434147, 48) df_generator
(434145, 37) df_hydraulic
(434145, 48) df_gen_bear
(434145, 39) df_transformer
(434145, 36) df_gearbox


In [17]:
for i, key in enumerate(comp_prep_df_dict):
    comp_prep_df_dict[key] = comp_prep_df_dict[key].drop(
        columns=feat_drop_list[i])

In [18]:
for key in comp_prep_df_dict:
    print(comp_prep_df_dict[key].shape, key)

(434147, 31) df_generator
(434145, 26) df_hydraulic
(434145, 31) df_gen_bear
(434145, 28) df_transformer
(434145, 26) df_gearbox


In [19]:
for key in comp_prep_df_dict:
    comp_prep_df_dict[key] = group_por_frequency(comp_prep_df_dict[key],
                                                 period='Dia')

In [20]:
for key in comp_prep_df_dict:
    print(comp_prep_df_dict[key].shape, key)

(3042, 31) df_generator
(3041, 19) df_hydraulic
(3041, 18) df_gen_bear
(3041, 19) df_transformer
(3041, 16) df_gearbox


In [21]:
cols = ['Turbine_ID', 'Date', 'TTF', '60_days']

In [22]:
def add_features2(df_in, rolling_win_size=15):
    """Add rolling average and rolling standard deviation for sensors readings using fixed rolling window size.
    """
    cols =['Turbine_ID', 'Date', 'TTF', '60_days']
    other_cols = []
    for i in df_in.columns:
        if i not in cols:
            other_cols.append(i)
    all_cols = cols + other_cols
    
    df_in = df_in[all_cols]
    
    sensor_cols = []
    for i in df_in.columns[4:]:
        sensor_cols.append(i)

    sensor_av_cols = [nm+'_av' for nm in sensor_cols]
    sensor_sd_cols = [nm+'_sd' for nm in sensor_cols]

    df_out = pd.DataFrame()

    ws = rolling_win_size

    #calculate rolling stats for each engine id

    for m_id in pd.unique(df_in.Turbine_ID):

        # get a subset for each engine sensors
        df_engine = df_in[df_in['Turbine_ID'] == m_id]
        df_sub = df_engine[sensor_cols]

        # get rolling mean for the subset
        av = df_sub.rolling(ws, min_periods=1).mean()
        av.columns = sensor_av_cols

        # get the rolling standard deviation for the subset
        sd = df_sub.rolling(ws, min_periods=1).std().fillna(0)
        sd.columns = sensor_sd_cols

        # combine the two new subset dataframes columns to the engine subset
        new_ftrs = pd.concat([df_engine,av,sd], axis=1)

        # add the new features rows to the output dataframe
        df_out = pd.concat([df_out,new_ftrs])
    df_out = df_out.sort_values(by=['Turbine_ID', 'Date']   )
    return df_out

In [23]:
for key in comp_prep_df_dict:
    comp_prep_df_dict[key] = add_features2(comp_prep_df_dict[key], rolling_win_size=10)

In [24]:
for key in comp_prep_df_dict:
    print(comp_prep_df_dict[key].shape, key)

(3042, 85) df_generator
(3041, 49) df_hydraulic
(3041, 46) df_gen_bear
(3041, 49) df_transformer
(3041, 40) df_gearbox


In [25]:
df_train_comp_dict = {}
df_test_comp_dict = {}
for key in comp_prep_df_dict:
    df_train_comp_dict[key] = prepare_train_df(comp_prep_df_dict[key], meses = 3)
    df_test_comp_dict[key] = prepare_test_df(comp_prep_df_dict[key], meses = 3)

In [26]:
for key in df_train_comp_dict:
    print(df_train_comp_dict[key].shape, key)
for key in df_test_comp_dict:
    print(df_test_comp_dict[key].shape, key)

(2577, 85) df_generator
(2576, 49) df_hydraulic
(2576, 46) df_gen_bear
(2576, 49) df_transformer
(2576, 40) df_gearbox
(465, 85) df_generator
(465, 49) df_hydraulic
(465, 46) df_gen_bear
(465, 49) df_transformer
(465, 40) df_gearbox


In [27]:
for key in df_train_comp_dict:
    print(df_train_comp_dict[key].shape, key)
for key in df_test_comp_dict:
    print(df_test_comp_dict[key].shape, key)

(2577, 85) df_generator
(2576, 49) df_hydraulic
(2576, 46) df_gen_bear
(2576, 49) df_transformer
(2576, 40) df_gearbox
(465, 85) df_generator
(465, 49) df_hydraulic
(465, 46) df_gen_bear
(465, 49) df_transformer
(465, 40) df_gearbox


In [28]:
print('012 - separar entre x_train, x_test')
x_train = df_train_comp_dict.copy()
y_train = df_train_comp_dict.copy()
x_test = df_test_comp_dict.copy()
y_test = df_test_comp_dict.copy()

012 - separar entre x_train, x_test


In [29]:
print('xxx - transformar o y de float para int')
col_to_mantain_test = ['60_days']
for key in y_train:
    y_train[key] = y_train[key][col_to_mantain_test].to_numpy().astype(int)[:,0]
    y_test[key] = y_test[key][col_to_mantain_test].to_numpy().astype(int)[:,0]

xxx - transformar o y de float para int


In [30]:
for key in x_train:
    print(x_train[key].shape, key)
for key in y_train:
    print(y_train[key].shape, key)
for key in x_test:
    print(x_test[key].shape, key)
for key in y_test:
    print(y_test[key].shape, key)

(2577, 85) df_generator
(2576, 49) df_hydraulic
(2576, 46) df_gen_bear
(2576, 49) df_transformer
(2576, 40) df_gearbox
(2577,) df_generator
(2576,) df_hydraulic
(2576,) df_gen_bear
(2576,) df_transformer
(2576,) df_gearbox
(465, 85) df_generator
(465, 49) df_hydraulic
(465, 46) df_gen_bear
(465, 49) df_transformer
(465, 40) df_gearbox
(465,) df_generator
(465,) df_hydraulic
(465,) df_gen_bear
(465,) df_transformer
(465,) df_gearbox


In [31]:
print('012 - separar entre x_train, x_test')
# retirar as colunas que não devem entrar no x
cols_to_drop_train = ['Date', 'TTF', '60_days']
for key in x_train:
    x_train[key] = x_train[key].drop(columns=cols_to_drop_train)
    x_test[key] = x_test[key].drop(columns=cols_to_drop_train)

012 - separar entre x_train, x_test


In [32]:
for key in x_train:
    print(x_train[key].shape, key)
for key in x_test:
    print(x_test[key].shape, key)

(2577, 82) df_generator
(2576, 46) df_hydraulic
(2576, 43) df_gen_bear
(2576, 46) df_transformer
(2576, 37) df_gearbox
(465, 82) df_generator
(465, 46) df_hydraulic
(465, 43) df_gen_bear
(465, 46) df_transformer
(465, 37) df_gearbox


In [43]:
x_train['df_generator'][x_train['df_generator']['Turbine_ID']=='T01']['Blds_PitchAngle_Std'].std()

0.8012785469822229

In [44]:
print('014 - fazer o fit do CustomTransfMiguel para cada componente à pedreiro')
print('014.1 - df_generator')
scaler_df_generator = Cscl()
scaler_df_generator.fit(x_train['df_generator'])
# joblib.dump(scaler_df_generator, 'scaler_df_generator.joblib')
print('014.2 - df_hydraulic')
scaler_df_hydraulic = Cscl()
scaler_df_hydraulic.fit(x_train['df_hydraulic'])
# joblib.dump(scaler_df_hydraulic, 'scaler_df_hydraulic.joblib')
print('014.3 - df_gen_bear')
scaler_df_gen_bear = Cscl()
scaler_df_gen_bear.fit(x_train['df_gen_bear'])
# joblib.dump(scaler_df_gen_bear, 'scaler_df_gen_bear.joblib')
print('014.4 - df_transformer')
scaler_df_transformer = Cscl()
scaler_df_transformer.fit(x_train['df_transformer'])
# joblib.dump(scaler_df_transformer, 'scaler_df_transformer.joblib')
print('014.5 - df_gearbox')
scaler_df_gearbox = Cscl()
scaler_df_gearbox.fit(x_train['df_gearbox'])
# joblib.dump(scaler_df_gearbox, 'scaler_df_gearbox.joblib')

014 - fazer o fit do CustomTransfMiguel para cada componente à pedreiro
014.1 - df_generator
014.2 - df_hydraulic
014.3 - df_gen_bear
014.4 - df_transformer
014.5 - df_gearbox


CustomStandardScaler()

x_train['df_generator'].columns[9]

x_train['df_generator']['Blds_PitchAngle_Std'].isna().sum()

In [45]:
print('012.1 - considerar no y set apenas as turbinas que tiveram falhas')
train_turbines = {
    'df_generator': ['T11', 'T06'],
    'df_hydraulic': ['T06', 'T11'],
    'df_gen_bear': ['T07', 'T09'],
    'df_transformer': ['T07'],
    'df_gearbox': ['T09']
}
for key in df_train_comp_dict:
    df_train_comp_dict[key] = df_train_comp_dict[key][
        df_train_comp_dict[key]['Turbine_ID'].isin(train_turbines[key])]

012.1 - considerar no y set apenas as turbinas que tiveram falhas


x_train1 = {}
x_test1 = {}

In [53]:
def scale(X_train, X_test):
    X_train1 = X_train.loc[X_train['Turbine_ID']=='T01']
    X_test1 = X_test.loc[X_test['Turbine_ID']=='T01']
    X_train1 = X_train1.drop(columns='Turbine_ID')
    X_test1 = X_test1.drop(columns='Turbine_ID')
    
    sc = StandardScaler()
    X_train1 = sc.fit_transform(X_train1)
    X_test1 = sc.transform(X_test1)
    turbines = ['T06', 'T07', 'T09', 'T11']
    for turbine in turbines:
        X_train_ = X_train.loc[X_train['Turbine_ID']==turbine]
        X_test_ = X_test.loc[X_test['Turbine_ID']==turbine]
        X_train_ = X_train_.drop(columns='Turbine_ID')
        X_test_ = X_test_.drop(columns='Turbine_ID')
     
        sc = StandardScaler()
        X_train_ = sc.fit_transform(X_train_)
        X_test_ = sc.transform(X_test_)
        X_train1 = np.concatenate((X_train1, X_train_))
        X_test1 = np.concatenate((X_test1, X_test_))
    return X_train1, X_test1

In [ ]:
def scale(X_train, X_test):
    X_train1 = X_train.loc[X_train['Turbine_ID']=='T01']
    X_test1 = X_test.loc[X_test['Turbine_ID']=='T01']
    X_train1 = X_train1.drop(columns='Turbine_ID')
    X_test1 = X_test1.drop(columns='Turbine_ID')
    
    sc = StandardScaler()
    X_train1 = sc.fit_transform(X_train1)
    X_test1 = sc.transform(X_test1)
    turbines = ['T06', 'T07', 'T09', 'T11']
    for turbine in turbines:
        X_train_ = X_train.loc[X_train['Turbine_ID']==turbine]
        X_test_ = X_test.loc[X_test['Turbine_ID']==turbine]
        X_train_ = X_train_.drop(columns='Turbine_ID')
        X_test_ = X_test_.drop(columns='Turbine_ID')
     
        sc = StandardScaler()
        X_train_ = sc.fit_transform(X_train_)
        X_test_ = sc.transform(X_test_)
        X_train1 = np.concatenate((X_train1, X_train_))
        X_test1 = np.concatenate((X_test1, X_test_))
    return X_train1, X_test1

def fit2(X, y=None):
    mean_ = {}
    std_ = {}

    for turbine in ['T01','T06','T07','T09','T11']:
        X_filter = X[X['Turbine_ID']==turbine]
        X_filter = X_filter.drop(columns='Turbine_ID').to_numpy()
        mean_[turbine] = X_filter.mean(axis=0)
        std_[turbine] = X_filter.std(axis=0)

    return mean_,std_

def transform2(X, y=None):
    X_turbine={}

    for turbine in ['T01','T06','T07','T09','T11']:
        if std_[turbine] == 0:
            X_turbine[turbine] = (X[X['Turbine_ID']==turbine].drop(columns='Turbine_ID').to_numpy() - mean_[turbine]) / 1
        else:
            X_turbine[turbine] = (X[X['Turbine_ID']==turbine].drop(columns='Turbine_ID').to_numpy() - mean_[turbine]) / std_[turbine]

    X_turbine_all=X_turbine['T01']

    for turbine in ['T06','T07','T09','T11']:

        X_turbine_all=np.concatenate([X_turbine_all,X_turbine[turbine]])

    return X_turbine_all

mean_,std_ = fit2(x_train['df_generator'])

std_['T01']

transform(x_train['df_generator'])

In [46]:
print('016 - aplicar o scale com o fit efectuado no treino')
x_train['df_generator'] = scaler_df_generator.transform(x_train['df_generator'])
x_train['df_hydraulic'] = scaler_df_hydraulic.transform(x_train['df_hydraulic'])
x_train['df_gen_bear'] = scaler_df_gen_bear.transform(x_train['df_gen_bear'])
x_train['df_transformer'] = scaler_df_transformer.transform(x_train['df_transformer'])
x_train['df_gearbox'] = scaler_df_gearbox.transform(x_train['df_gearbox'])

016 - aplicar o scale com o fit efectuado no treino


C:\Users\ricar\code\RicardoJPedro82\wmillfailprev\wmillfailprev\c_model_related.py:32: RuntimeWarning: invalid value encountered in true_divide
  X_turbine[turbine] = (X[X['Turbine_ID']==turbine].drop(columns='Turbine_ID').to_numpy() - self.mean_[turbine]) / self.std_[turbine]


In [47]:
print('016 - aplicar o scale com o fit efectuado no teste')
x_test['df_generator'] = scaler_df_generator.transform(x_test['df_generator'])
x_test['df_hydraulic'] = scaler_df_hydraulic.transform(x_test['df_hydraulic'])
x_test['df_gen_bear'] = scaler_df_gen_bear.transform(x_test['df_gen_bear'])
x_test['df_transformer'] = scaler_df_transformer.transform(x_test['df_transformer'])
x_test['df_gearbox'] = scaler_df_gearbox.transform(x_test['df_gearbox'])

016 - aplicar o scale com o fit efectuado no teste


C:\Users\ricar\code\RicardoJPedro82\wmillfailprev\wmillfailprev\c_model_related.py:32: RuntimeWarning: divide by zero encountered in true_divide
  X_turbine[turbine] = (X[X['Turbine_ID']==turbine].drop(columns='Turbine_ID').to_numpy() - self.mean_[turbine]) / self.std_[turbine]


In [48]:
for key in x_train:
    print(x_train[key].shape, key)
for key in x_test:
    print(x_test[key].shape, key)

(2577, 81) df_generator
(2576, 45) df_hydraulic
(2576, 42) df_gen_bear
(2576, 45) df_transformer
(2576, 36) df_gearbox
(465, 81) df_generator
(465, 45) df_hydraulic
(465, 42) df_gen_bear
(465, 45) df_transformer
(465, 36) df_gearbox


exp1 = pd.DataFrame(x_test['df_generator'])

x_test['df_generator'][9]

exp1[9]

In [49]:
print('017 - Instanciar e treinar o modelo correspondente')
generator_model = tr(x_train=x_train['df_generator'],
                     y_train=y_train['df_generator'],
                     x_test=x_test['df_generator'],
                     component='df_generator')
generator_model.train()
# joblib.dump(generator_model, 'generator_model.joblib')

017 - Instanciar e treinar o modelo correspondente


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
print('018 - Obter a previsão para a métrica de poupança')
y_pred_generator = generator_model.predict()